In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout,BatchNormalization

In [ ]:
columns = ['person', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11',\
       'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21',\
       'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31',\
       'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41',\
       'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51',\
       'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'O']

In [ ]:
max_rows, max_cols = 1034504, 59
#max_rows, max_cols = 884009, 59

In [ ]:
# Pad the time series to the same length
x_padded = []
y_padded = []
for i in range(1,12):
    directory = '/kaggle/input/merged-data/' + str("{:03d}".format(i) + '.csv')
    df_chunk = pd.read_csv(directory).values
    
    x_padding = np.pad(df_chunk[:, :-1], ((0, max_rows - df_chunk.shape[0]), (0, 0)), 'constant')
    y_padding = np.pad(df_chunk[:, -1], ((0, max_rows - df_chunk.shape[0])), 'constant')

    
    print(df_chunk.shape, x_padding.shape, y_padding.shape)
    x_padded.append(x_padding)
    y_padded.append(y_padding)

x_padded = np.array(x_padded)
y_padded = np.array(y_padded)

In [ ]:
sequence_length = 10 
num_features = 59

num_sequences = len(x_padded) // sequence_length
total_samples = num_sequences * sequence_length

xpadded = x_padded[:total_samples]
ypadded = y_padded[:total_samples]

X = xpadded.reshape(-1, sequence_length, num_features)
y = ypadded.reshape(-1, sequence_length)
X.shape, y.shape

In [ ]:
#model
i = Input(shape=(10, 59))
x = LSTM(128, return_sequences = True)(i)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = LSTM(64, return_sequences = True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)


'''x = LSTM(128, return_sequences = True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)'''

x = Dense(1, activation = 'sigmoid')(x)

model = Model(i, x)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
r = model.fit(X, y, epochs = 50, batch_size = 256, validation_split = 0.2, callbacks=[early_stopping])

In [ ]:
plt.plot(r.history['accuracy'], label = 'accuracy')
plt.plot(r.history['val_accuracy'], label = 'val - accuracy')
plt.legend(loc = 'best')

In [ ]:
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val - loss')
plt.legend(loc = 'best')

In [ ]:
test_data = pd.read_csv('/kaggle/input/merged-data/012.csv').values
x_test_padded = np.pad(test_data[:, :-1], ((0, max_rows - test_data.shape[0]), (0,0)), 'constant')
y_test_padded = np.pad(test_data[:, -1], ((0, max_rows - test_data.shape[0])), 'constant')
                      
print(x_test_padded.shape, y_test_padded.shape)

In [ ]:
num_sequences = len(x_test_padded) // sequence_length
total_samples = num_sequences * sequence_length

x_testpadded = x_test_padded[:total_samples]
y_testpadded = y_test_padded[:total_samples]

X_test = x_testpadded.reshape(-1, sequence_length, num_features)
y_test = y_testpadded.reshape(-1, sequence_length)
X_test.shape, y_test.shape

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = y_pred.reshape(-1, )
y_test = y_test.reshape(-1,)

In [ ]:
y_pred_bool = [0 if x < 0.7 else 1 for x in y_pred]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test.reshape(-1,), y_pred_bool)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred_bool)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()